# Customer Support Planning Pattern
## Using AI to Plan and Handle Customer Support Tickets
### Introduction
This notebook shows how to use the planning pattern for customer support scenarios. We'll create system that:
- Analyzes customer inquiries
- Creates step-by-step support plans
- Tracks resolution progress
- Adapts to changing ticket priorities

## 1. Initial Setup
First, let's set up our environment and create some sample data.

In [1]:
import os
from openai import OpenAI
import json
from datetime import datetime

# Initialize the MaLLaM client
mallam_api_key = os.getenv("MALLAM_API_KEY")

client = OpenAI(
    base_url="https://api.mesolitica.com",
    api_key=mallam_api_key
)

# sample customer info db
CUSTOMER_DB = {
    "cust101": {
        "name": "Akmal Firdaus",
        "plan": "premium",
        "language": "english",
        "history": ["billing_issue_resolved", "technical_support_pending"]
    }
}

# sample knowledge base
KNOWLEDGE_BASE = {
    "login": "Standard login troubleshooting steps...",
    "billing": "Billing resolution procedures...",
    "technical": "Technical support guidelines..."
}

## 2. Analyze Customer Inquiry
First, let's analyze the customers's messag to understand what they need.

In [2]:
# sample support ticket
support_ticket = {
    "ticket_id": "T789",
    "customer_id": "cust101",
    "subject": "Cannot accesss prmium features",
    "message": """Hi, I've been trying to access the premium features I paid for, but keep getting
    an error message. I upgraded my account yesterday but still can't access anything. Please help!""",
    "priority": "high",
    "timestamp": datetime.now().isoformat()
}

# analysis prompt
analysis_prompt = [
    {
        "role": "system",
        "content": """You are a support ticket analyzer. For each ticket:
        1. Identify the main issue
        2. Determine urgency
        3. Categorize the problem
        4. List required information
        Return analysis as JSON with:
        - issue_type
        - urgency_level (1-5)
        - category
        - required_info: array of needed information
        - initial_assessment"""
    },
    {
        "role": "user",
        "content": f"""Customer: {CUSTOMER_DB[support_ticket['customer_id']]['name']}
        Plan: {CUSTOMER_DB[support_ticket['customer_id']]['plan']}
        Message: {support_ticket['message']}"""
    }
]

# get the analysis
analysis_response = client.chat.completions.create(
    model="mallam-small",
    messages=analysis_prompt,
    max_tokens=1024
)

ticket_analysis = json.loads(analysis_response.choices[0].message.content)
print("\nTicket Analysis:")
print(json.dumps(ticket_analysis, indent=2))


Ticket Analysis:
{
  "issue_type": "Access Issues",
  "urgency_level": 3,
  "category": "Subscription and Billing",
  "required_info": [
    "subscription plan",
    "error message",
    "date of upgrade"
  ],
  "initial_assessment": "The customer is experiencing difficulty accessing premium features despite upgrading their subscription. The issue might be related to the subscription not being activated properly or a system glitch."
}


## 3. Create Support Resolution Plan
Based on the analysis, let's create a step-by-step plan to resolve the issue

In [3]:
# planning prompt
planning_prompt = [
    {
        "role": "system",
        "content": """You are a support planning assistant. Create a detailed plan to resolve the ticket.
        Return plan as JSON with:
        - steps: array of resolution steps containing:
            - step_number
            - action
            - required_tools
            - expected_outcome
            - estimated_time (minutes)
        - fallback_steps: array of alternative steps if initial plan fails
        - escalation_criteria: when to escalate to human/senior support"""
    },
    {
        "role": "user",
        "content": f"""Ticket Analysis: {json.dumps(ticket_analysis)}
        Customer info: {json.dumps(CUSTOMER_DB[support_ticket['customer_id']])}"""
    }
]

# get the support plan
plan_response = client.chat.completions.create(
    model="mallam-small",
    messages=planning_prompt,
    max_tokens=1024
)

support_plan = json.loads(plan_response.choices[0].message.content)
print(support_plan)

print("\nSupport Resolution Plan:")
print(json.dumps(support_plan, indent=2))

{'steps': [{'step_number': 1, 'action': 'Confirm that the subscription has been upgraded and activated successfully.', 'required_tools': [], 'expected_outcome': "Verify that the subscription upgrade was processed correctly and that the customer's account reflects the new plan.", 'estimated_time': 5}, {'step_number': 2, 'action': 'Check if the customer has any pending payments or unresolved billing issues.', 'required_tools': [], 'expected_outcome': 'Determine whether there are any outstanding payments or billing discrepancies that could prevent access to premium features.', 'estimated_time': 5}, {'step_number': 3, 'action': 'Review system logs for any errors related to the subscription activation process.', 'required_tools': ['access to system logs'], 'expected_outcome': 'Identify any technical issues or errors in the subscription activation process that may have caused the access problems.', 'estimated_time': 10}, {'step_number': 4, 'action': 'Reach out to the customer to gather more 

## 4. Track Resolution Progress
Let's implement progress tracking for our support plan.

In [4]:
# create a tracking dictionary
resolution_progress = {
    "ticket_id": support_ticket["ticket_id"],
    "start_time": datetime.now().isoformat(),
    "total_steps": len(support_plan["steps"]),
    "completed_steps": 0,
    "current_step": 1,
    "status": "in_progress",
    "step_status": {},
    "notes": []
}

# initialize step status
for step in support_plan["steps"]:
    resolution_progress["step_status"][str(step["step_number"])] = {
        "status": "pending",
        "start_time": None,
        "completion_time": None,
        "outcome": None
    }
    
# Example: update progress for a step
def update_step_progress(step_number, status, outcome=None):
    step = str(step_number)
    if step in resolution_progress["step_status"]:
        resolution_progress["step_status"][step].update({
            "status": status,
            "completion_time": datetime.now().isoformat(),
            "outcome": outcome
        })
        if status == "completed":
            resolution_progress["completed_steps"] += 1
            resolution_progress["current_step"] = step_number + 1
            
# example usage
update_step_progress(1, "completed", "Successfully verified account status")
print("\nUpdated Progress:")
print(json.dumps(resolution_progress, indent=2))


Updated Progress:
{
  "ticket_id": "T789",
  "start_time": "2025-01-16T03:16:27.777458",
  "total_steps": 5,
  "completed_steps": 1,
  "current_step": 2,
  "status": "in_progress",
  "step_status": {
    "1": {
      "status": "completed",
      "start_time": null,
      "completion_time": "2025-01-16T03:16:27.777740",
      "outcome": "Successfully verified account status"
    },
    "2": {
      "status": "pending",
      "start_time": null,
      "completion_time": null,
      "outcome": null
    },
    "3": {
      "status": "pending",
      "start_time": null,
      "completion_time": null,
      "outcome": null
    },
    "4": {
      "status": "pending",
      "start_time": null,
      "completion_time": null,
      "outcome": null
    },
    "5": {
      "status": "pending",
      "start_time": null,
      "completion_time": null,
      "outcome": null
    }
  },
  "notes": []
}


## 5. Handle Plan Adaptation
Sometimes we need to adapt our plan based on new information or failed steps.

In [5]:
# example: Need to adapt plan due to failed step
trigger = {
    "step_number": 2,
    "reason": "Customer's account shows payment pending",
    "new_information": "Payment processing delay detected"
}

adaptation_messages = [
    {
        "role": "system",
        "content": """You are a support plan adapter. When a step fails:
        1. Analyze the failure reason
        2. Modify the remaining plan steps
        3. Add any necessary preliminary steps
        Return modified plan in same JSON format"""
    },
    {
        "role": "user",
        "content": f"""Current Plan: {json.dumps(support_plan)}
        Failed step: {json.dumps(trigger)}"""
    }
]

# get adapted plan
adaptation_response = client.chat.completions.create(
    model="mallam-small",
    messages=adaptation_messages,
    max_tokens=1024
)

adapted_plan = json.loads(adaptation_response.choices[0].message.content)
print(adapted_plan)

print("\nAdapted Support Plan:")
print(json.dumps(adapted_plan, indent=2))

{'steps': [{'step_number': 1, 'action': 'Confirm that the subscription has been upgraded and activated successfully.', 'required_tools': [], 'expected_outcome': "Verify that the subscription upgrade was processed correctly and that the customer's account reflects the new plan.", 'estimated_time': 5}, {'step_number': 2, 'action': 'Check if the customer has any pending payments or unresolved billing issues.', 'required_tools': [], 'expected_outcome': 'Determine whether there are any outstanding payments or billing discrepancies that could prevent access to premium features.', 'estimated_time': 5, 'status': 'failed', 'failure_reason': "Customer's account shows payment pending", 'additional_steps': [{'step_number': 7, 'action': 'Communicate with the payment provider to check on the status of the pending payment.', 'required_tools': ["access to payment provider's system"], 'expected_outcome': 'Get an update on when the payment will be processed and cleared.', 'estimated_time': 10}, {'step_n

In [9]:
update_step_progress(2, "completed", "Customer's account shows payment pending")
print("\nUpdated Progress:")
print(json.dumps(resolution_progress, indent=2))


Updated Progress:
{
  "ticket_id": "T789",
  "start_time": "2025-01-16T03:16:27.777458",
  "total_steps": 5,
  "completed_steps": 2,
  "current_step": 3,
  "status": "in_progress",
  "step_status": {
    "1": {
      "status": "completed",
      "start_time": null,
      "completion_time": "2025-01-16T03:16:27.777740",
      "outcome": "Successfully verified account status"
    },
    "2": {
      "status": "completed",
      "start_time": null,
      "completion_time": "2025-01-16T03:23:03.400373",
      "outcome": "Customer's account shows payment pending"
    },
    "3": {
      "status": "pending",
      "start_time": null,
      "completion_time": null,
      "outcome": null
    },
    "4": {
      "status": "pending",
      "start_time": null,
      "completion_time": null,
      "outcome": null
    },
    "5": {
      "status": "pending",
      "start_time": null,
      "completion_time": null,
      "outcome": null
    }
  },
  "notes": []
}


## 6. Generate Customer Updates
Keep the customer informed about progress

In [10]:
# update prompt
update_message = [
    {
        "role": "system",
        "content": """You are a customer communication assistant. Create updates that are:
        - clear and professional
        - specific about progress
        - informative about next steps
        Return message in JSON with:
        - subject
        - message
        - next_steps
        - estimated_completion"""
    },
    {
        "role": "user",
        "content": f"""Progress: {json.dumps(resolution_progress, indent=2)}
        Customer info: {json.dumps(CUSTOMER_DB[support_ticket['customer_id']])}"""
    }
]

# generate customer update
update_response = client.chat.completions.create(
    model="mallam-small",
    messages=update_message,
    max_tokens=512
)

customer_update = json.loads(update_response.choices[0].message.content)
print(customer_update)

print("\nCustomer Update:")
print(json.dumps(customer_update, indent=2))

{'subject': 'Update on Billing Issue Resolution for Akmal Firdaus', 'message': 'Dear Akmal Firdaus,\n\nThank you for reaching out to us regarding your billing issue. Our team is actively working on resolving this matter.\n\nWe have already completed two steps in our process:\n1. Verified your account status, which has been successfully confirmed.\n2. Noted that your account shows a payment pending status.\n\nCurrently, we are proceeding to the third step, which involves investigating the reason for the pending payment. We will be reviewing your transaction history and verifying any recent payments made.\n\nOnce we have gathered the necessary information, we will take the appropriate actions to resolve the pending payment and ensure that your billing is up to date.\n\nShould you have any additional questions or require further assistance during this process, please do not hesitate to contact us.\n\nSincerely,\nThe Support Team', 'next_steps': 'Proceeding to the third step: Investigating